In [3]:
import pandas as pd
import nltk
import spacy
import gensim
import seaborn as sb
from gensim import corpora, models, similarities
from spacy.lang.en import English
from nltk.corpus import wordnet as wn

In [4]:
data=pd.read_csv("DurhamCityOriginalDataFY20.csv",)

In [5]:
data.head()

,Unnamed: 0,page_number,word
0,1,1,fiscal
1,2,1,year
2,3,1,adopted
3,4,1,budget
4,5,1,july


In [8]:
data2=data.drop(data.columns[0], axis=1)

In [10]:
data2.describe()

,page_number
count,81524.000000
mean,192.937393
std,108.890666
min,1.000000
25%,108.000000
50%,202.000000
75%,276.000000
max,398.000000


In [11]:
#cleaning 
spacy.load('en')
parser=English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.orth_.startswith('http'):
            lda_tokens.append('com')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [12]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
def get_lemma(word):
    lemma=wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [14]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [15]:
#filtering stopwords
nltk.download('stopwords')
stop=set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
#preparing for lda
def prepare_lda(text):
    tokens=tokenize(text)
    tokens=[token for token in tokens if len(token)>4]
    tokens=[token for token in tokens if token not in stop]
    tokens=[get_lemma(token) for token in tokens]
    return tokens

In [17]:
#creating tokens
import random
text_data=[]
with open('DurhamCityOriginalDataFY20.csv') as f:
    for line in f:
        tokens=prepare_lda(line)
        if random.random()>.99:
            print(tokens)
            text_data.append(tokens)

['73",3,"organization']
['148",4,"manager']
['239",4,"hispanic']
['281",4,"initiatives']
['296",4,"assistant']
['356",5,"meets']
['368",5,"a']
['389",6,"rehberg']
['480",7,"of']
['510",7,"summaryall']
['591",7,"parking']
['599",7,"center']
['747",8,"stormwater']
['949",10,"to']
['1219",10,"also']
['1391",10,"ordinance']
['1467",11,"or']
['1636",11,"statutes']
['1884",11,"financial']
['1945",11,"either']
['2058",12,"is']
['2526",13,"will']
['2538",13,"been']
['2576",13,"management']
['2688",13,"housing']
['2868",14,"for']
['2883",14,"reach']
['3030",14,"this']
['3108",14,"done']
['3124",14,"of']
['3247",14,"the']
['3317",15,"the']
['3373",15,"assessed']
['3424",15,"of']
['3517",15,"following']
['3623",16,"funding']
['3876",16,"as']
['3956",17,"fund']
['3958",17,"the']
['3959",17,"revenueneutral']
['4016",17,"cents']
['4116",17,"house']
['4216",18,"occupancy']
['4224",18,"government']
['4359",19,"neutral']
['4368",19,"increase']
['4556",20,"will']
['4703",21,"to']
['4818",21,"and']
['487

['37772",191,"a']
['37858",192,"enhanced']
['37993",192,"the']
['38155",192,"davis']
['38316",193,"as']
['38392",193,"potholes']
['38535",193,"review']
['38707",195,"public']
['38708",195,"works']
['38808",196,"and']
['38840",196,"the']
['38854",196,"development']
['39018",196,"of']
['39058",196,"certain']
['39081",196,"and']
['39150",196,"city']
['39155",196,"plans']
['39184",196,"advisory']
['39193",196,"durham']
['39219",197,"through']
['39271",197,"approvals']
['39311",197,"time']
['39315",197,"revenues']
['39380",198,"court']
['39537",198,"three']
['39657",198,"problem']
['39711",198,"including']
['39743",198,"to']
['39923",199,"presented']
['40032",199,"meetings']
['40040",199,"and']
['40055",199,"in']
['40428",200,"community']
['40487",200,"temporary']
['40538",200,"metropolitan']
['40659",201,"fy']
['40924",203,"fte']
['41167",204,"initiatives']
['41541",205,"shelter']
['41632",206,"was']
['41944",206,"fy']
['42029",206,"to']
['42084",206,"end']
['42192",207,"on']
['42225",207,

['74601",354,"the']
['74973",362,"process']
['74974",362,"begins']
['75066",362,"appropriate']
['75298",362,"becomes']
['75352",362,"and']
['75448",383,"county']
['75460",383,"assets']
['75532",383,"station']
['75553",383,"incorporated']
['75701",383,"work']
['75866",384,"racial']
['75872",384,"the']
['75878",384,"under']
['75954",384,"the']
['76092",384,"pm']
['76121",384,"of']
['76365",385,"among']
['76394",385,"environmental']
['76785",387,"to']
['76917",387,"offering']
['77090",387,"science']
['77135",387,"the']
['77167",387,"carolina']
['77511",388,"with']
['77615",389,"average']
['77705",389,"chapel']
['77734",389,"lots']
['77942",389,"onemillion']
['78017",389,"site']
['78019",389,"open']
['78033",389,"and']
['78053",389,"of']
['78134",390,"of']
['78140",390,"district']
['78202",390,"in']
['78499",390,"is']
['78656",390,"and']
['78922",391,"fee']
['79137",391,"road']
['79182",391,"anchor']
['79219",391,"eno']
['79256",391,"of']
['79553",392,"delivering']
['79619",392,"approval']

In [18]:
#creating dictionary
from gensim import corpora
dictionary=corpora.Dictionary(text_data)
corpus= [dictionary.doc2bow(text) for text in text_data]
import pickle 
pickle.dump(corpus, open('corpus.pkl', 'wb'))

In [19]:
#example
ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=6)
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} word: {}'.format(idx, topic))

Topic: 0 word: 0.007*"4556",20,"will" + 0.007*"57058",264,"upgrades" + 0.007*"60856",276,"rubber" + 0.007*"70265",307,"customers" + 0.007*"53399",250,"until" + 0.007*"76394",385,"environmental" + 0.007*"67620",299,"pavement" + 0.007*"36723",184,"and" + 0.007*"4703",21,"to" + 0.007*"13447",68,"the"
Topic: 1 word: 0.006*"49899",237,"terrorist" + 0.006*"34265",171,"reports" + 0.006*"60092",274,"johnson" + 0.006*"13217",67,"grants" + 0.006*"81064",396,"the" + 0.006*"10615",47,"job" + 0.006*"70303",307,"satisfaction" + 0.006*"74040",322,"use" + 0.006*"45047",215,"partners" + 0.006*"26770",131,"balance"
Topic: 2 word: 0.007*"46864",222,"to" + 0.007*"28284",141,"programs" + 0.007*"49968",238,"meetings" + 0.007*"69255",305,"north" + 0.007*"66387",295,"and" + 0.007*"48451",228,"for" + 0.007*"7168",27,"of" + 0.007*"62913",282,"actual" + 0.007*"39058",196,"certain" + 0.007*"45110",215,"involved"
Topic: 3 word: 0.007*"54318",251,"protocol" + 0.007*"77734",389,"lots" + 0.007*"59481",273,"course" + 

In [21]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)